In [ ]:
# Phemex USDT-M Balance Check (No Terminal)

import os, time, hmac, hashlib, json, urllib.parse, urllib.request
from pathlib import Path

# Load .env if present (simple parser, no external deps)
root = Path(__file__).resolve().parents[1]
env_path = root / '.env'
if env_path.exists():
    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        if '=' in line:
            k, v = line.split('=', 1)
            os.environ.setdefault(k.strip(), v.strip())

API_KEY = os.getenv('PHEMEX_API_KEY', '')
API_SECRET = os.getenv('PHEMEX_API_SECRET', '')
BASE_URL = os.getenv('PHEMEX_BASE_URL', 'https://api.phemex.com')
CURRENCY = os.getenv('PHEMEX_CURRENCY', 'USDT')

assert API_KEY and API_SECRET, 'Set PHEMEX_API_KEY and PHEMEX_API_SECRET in .env before running.'


def sign(secret: str, path: str, query: str, expiry: str, body: str) -> str:
    msg = f"{path}{query}{expiry}{body}"
    return hmac.new(secret.encode(), msg.encode(), hashlib.sha256).hexdigest()


def get_account_info(currency: str) -> dict:
    path = '/accounts/accountInfo'
    query = f"currency={urllib.parse.quote(currency)}"
    expiry = str(int(time.time()) + 60)
    body = ''
    signature = sign(API_SECRET, path, query, expiry, body)

    url = f"{BASE_URL}{path}?{query}"
    headers = {
        'x-phemex-access-token': API_KEY,
        'x-phemex-request-expiry': expiry,
        'x-phemex-request-signature': signature,
    }
    req = urllib.request.Request(url, headers=headers, method='GET')
    with urllib.request.urlopen(req, timeout=20) as resp:
        return json.loads(resp.read().decode('utf-8'))

# Run and pretty print
resp = get_account_info(CURRENCY)
print(json.dumps(resp, indent=2, sort_keys=True))


: 